In [1]:
import folium
import pandas as pd
import json
import os
from branca.colormap import linear, StepColormap

In [2]:
# Load the spreadsheet
file_path = 'allactive.xlsx'

# Load the SortedSummary sheet into a DataFrame
df_sorted_summary = pd.read_excel(file_path, sheet_name='Summary')

# Extract the ID and Count columns
employee_counts = df_sorted_summary[['ID', 'Count']]

# Function to prepend 'PH' or 'PH0' based on the length of the ID
def prepend_ph(id_value):
    id_str = str(id_value)
    if len(id_str) == 9:
        return 'PH' + id_str
    elif len(id_str) == 8:
        return 'PH0' + id_str
    else:
        return id_str

# Apply the function to the ID column
employee_counts.loc[:, 'ID'] = employee_counts['ID'].apply(prepend_ph)

# Display the first few rows of the modified DataFrame
print(employee_counts.head())

            ID  Count
0  PH137603004      4
1  PH137607021      1
2  PH137603001      8
3  PH137603006      4
4  PH137604007      3


In [3]:
# Initialization code
jsonfolder_path = ['/Users/jerritor2/Desktop/EMS_Data/Bus_Analysis/philippines-json-maps/2023/geojson/municities/hires',
                  '/Users/jerritor2/Desktop/EMS_Data/Bus_Analysis/philippines-json-maps/2019/geojson/barangays/hires']

m = folium.Map()

def generateMapLocation(mapfocusindex):
    global m
    
    if mapfocusindex == 0: # all ph
        m = folium.Map(location=[12.616870715056407, 122.34833708101706], zoom_start=5.5)
        print("Setting focus to all of Philippines")
    elif mapfocusindex == 1: # calabarzon + metro manila
        m = folium.Map(location=[14.468224838723954, 121.04553358573668], zoom_start=10)
        print("Setting focus to Calabarzon + Metro Manila")
    else: # calabarzon centered on calamba
        m = folium.Map(location=[14.274281440588128, 121.05943840821561], zoom_start=11)
        print("Setting focus to Calabarzon")

def generateMapLevel(fpathindex):
    if fpathindex == 0:
        print("Setting borders to 2023 Municipalities")
    else:
        print("Setting borders to 2019 Barangays")
    
    # Read and store GeoJSON data
    geojson_features = []
    for filename in os.listdir(jsonfolder_path[fpathindex]):
        if filename.endswith('.json'):
            file_path = os.path.join(jsonfolder_path[fpathindex], filename)
            with open(file_path, 'r') as file:
                data = json.load(file)
                geojson_features.extend(data['features'])
    
    print("GeoJSON data loaded.")
    return geojson_features

def generateMap(focusindex, pathindex):
    generateMapLocation(focusindex)
    geojson_features = generateMapLevel(pathindex)
    return geojson_features

# Load the GeoJSON data for barangays
geojson_features = generateMap(2, 1)

# Ensure ID column is string
employee_counts['ID'] = employee_counts['ID'].astype(str)

# Create a dictionary for quick lookup of employee counts by ADM4_PCODE
employee_count_dict = employee_counts.set_index('ID')['Count'].to_dict()

# Define bins and colormap
#colormap = linear.OrRd_09.scale(min(employee_count_dict.values()), max(employee_count_dict.values()))
bins = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 25, 27, 36, 41, 55]
colormap = StepColormap(
    colors=['white', '#ff0000'],  # transition from white to red
    index=bins,
    vmin=min(bins),
    vmax=max(bins),
    caption='Employee Count'
)

# Add all barangay borders with thin and transparent style
for feature in geojson_features:
    pcode = feature['properties']['ADM4_PCODE']
    if pcode in employee_count_dict:
        # Style for choropleth features
        feature['properties']['style'] = {
            'fillColor': '#%02x%02x%02x' % (255, 255 - employee_count_dict[pcode] * 5, 0), # Example color scaling
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7,
            'lineOpacity': 0.2
        }
        feature['properties']['highlight'] = {
            'weight': 3,
            'color': 'blue'
        }
    else:
        # Style for non-choropleth features
        feature['properties']['style'] = {
            'fillColor': 'transparent',
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0,
            'lineOpacity': 0.5
        }

# Add GeoJSON layer with custom styles
folium.GeoJson(
    {'type': 'FeatureCollection', 'features': geojson_features},
    name='barangay_borders',
    style_function=lambda x: x['properties']['style'],
    highlight_function=lambda x: x['properties'].get('highlight', {}),
    tooltip=folium.GeoJsonTooltip(
        fields=['ADM1_EN','ADM2_EN','ADM3_EN','ADM4_EN'],
        aliases=['Region: ','Province: ','Municipality: ','Barangay: '],
        localize=True
    )
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('allactive2.html')

Setting focus to Calabarzon
Setting borders to 2019 Barangays
GeoJSON data loaded.


/var/folders/hk/l0w5001d7pdc860p4ssywh3h0000gn/T/ipykernel_6552/3231424389.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  employee_counts['ID'] = employee_counts['ID'].astype(str)


In [ ]:
# Display the map in Jupyter Notebook (if running in such an environment)
m